In [1]:
import re
from functools import partial
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

def removeUnicode(text):
	#Removes unicode strings like "\u002c" and "x96"
	text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
	text = re.sub(r'[^\x00-\x7f]',r'',text)
	return text

def replaceURL(text):
	#Replaces url address with "url" 
	text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','<url>',text)
	text = re.sub(r'#([^\s]+)', r'\1', text)
	return text

def replaceAtUser(text):
	#Replaces "@user" with "atUser"
	text = re.sub('@[^\s]+','atUser',text)
	return text

def removeHashtag(text):
	#Removes hastag in front of a word
	text = re.sub(r'#([^\s]+)', r'\1', text)
	return text

def removeNumbers(text):
	#Removes integers
	text = ''.join([i for i in text if not i.isdigit()])         
	return text

def replaceMulExcl(text):
	#Replaces repetitions of exlamation marks
	text = re.sub(r"(\!)\1+", '!', text)
	return text

def replaceMulQues(text):
	#Replaces repetitions of question marks
	text = re.sub(r"(\?)\1+", '?', text)
	return text

def replaceMulStop(text):
	#Replaces repetitions of stop marks
	text = re.sub(r"(\.)\1+", '.', text)
	return text

def countMulExcl(text):
	#count repetitions of exlamation marks
	return len(re.findall(r"(\!)\1+", text))

def countMulQues(text):
	#Count repetitions of question marks
	return len(re.findall(r"(\?)\1+", text))

def countMulStop(text):
	#Count repetitions of stop marks
	return len(re.findall(r"(\.)\1+", text))

def countElongated(text):
	#count of how many words are elongated
	regex = re.compile(r"(.)\1{2}")
	return len([word for word in text.split() if regex.search(word)])

def countAllCaps(text):
	#count of how many words are all caps
	return len(re.findall("[A-Z0-9]{3,}", text))

#Creates a dictionary with slangs and their equivalents and replaces them
with open('slang.txt') as file:
	slang_map = dict(map(str.strip, line.partition('\t')[::2])
	for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True)
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)])

#punctuation list for replacing

puncts = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '$', '&', '/', '[', ']', '%', '=', '*', '+', '\\', '•',  '~', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def removePuncts(x):
	x = str(x)
	for punct in puncts:
		if punct in x:
			x = x.replace(punct, f' ')
	return x



def countSlang(text):
	# counts how many slang words and a list of found slangs
	slangCounter = 0
	slangsFound = []
	tokens = nltk.word_tokenize(text)
	for word in tokens:
		if word in slang_words:
			slangsFound.append(word)
			slangCounter += 1
	return slangCounter, slangsFound

#Replaces contractions from a string to their equivalents
contraction_patterns = [ (r'I\'m', 'I am'),(r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
						 (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
def replaceContraction(text):
	patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
	for (pattern, repl) in patterns:
		(text, count) = re.subn(pattern, repl, text)
	return text

def replaceElongated(word):
	#Replaces an elongated word with its basic form

	repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
	repl = r'\1\2\3'
	if wordnet.synsets(word):
		return word
	repl_word = repeat_regexp.sub(repl, word)
	if repl_word != word:      
		return replaceElongated(repl_word)
	else:       
		return repl_word

def removeEmoticons(text):
	#Removes emoticons from text 
	text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
	return text

def countEmoticons(text):
	#Input: a text, Output: how many emoticons
	return len(re.findall(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', text))


### Spell Correction begin ###
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('spell_correction.txt').read()))

def P(word, N=sum(WORDS.values())): 
	#P robability of `word`.
	return WORDS[word] / N

def spellCorrection(word): 
	#Most probable spelling correction for word.
	return max(candidates(word), key=P)

def candidates(word): 
	#Generate possible spelling corrections for word.
	return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
	#The subset of `words` that appear in the dictionary of WORDS.
	return set(w for w in words if w in WORDS)

def edits1(word):
	#All edits that are one edit away from `word`.
	letters    = 'abcdefghijklmnopqrstuvwxyz'
	splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
	deletes    = [L + R[1:]               for L, R in splits if R]
	transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
	replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
	inserts    = [L + c + R               for L, R in splits for c in letters]
	return set(deletes + transposes + replaces + inserts)

def edits2(word): 
	#All edits that are two edits away from `word`.
	return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Spell Correction End ###

In [2]:
import pandas as pd
import numpy as np
import string
from time import time
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 


def k_prep(inputText = 'null'):
	'''
	c = input("\n1.Enter or 2.default or 3.df ? :")

	if c == "2":
		text = "AFRICA,#AFRICANBAZE: asap goaaaaal @Breaking !!!! news:Nigeria oooooooh :-D aren't flag???? wont set ablaze..... 12000 in America. http://t.co/2nndBGwyEi,1"
		#print(text)
	
	elif c == "3" :
		text = inputText

	else:
		text = input("\nEnter the tweet: ")
	'''

	text = inputText
	#print("\nReplacing url from tweet\n")
	text = replaceURL(text)
	#print(text)

	#print("\nReplacing atuser from tweet\n")
	text = replaceAtUser(text)
	#print(text)

	#print("\nremoving hashtag in tweet\n")
	text = removeHashtag(text)
	#print(text)

	#print("\nreplace at user in tweet\n")
	text = replaceAtUser(text)
	#print(text)

	'''
	#print("\nremoving stopwords\n")
	#nltk.download('stopwords')
	from nltk.corpus import stopwords
	stop = set(stopwords.words('english'))

	d=[]
	d.append([x for x in text.split() if x not in stop])
	d = d[0]
	text = ' '.join(d)
	#print(text)
	'''

	#print("\nremove numbers from tweet\n")
	text = removeNumbers(text)
	#print(text)

	#print("\nremove emoticons from tweet\n")
	text = removeEmoticons(text)
	#print(text)

	#couting multple punctuations
	#print("\ncounting multiple punctuations\n")
	MultiExclMarks = 0
	MultiQuesMarks = 0
	MultiStopMarks = 0

	MultiExclMarks += countMulExcl(text)
	MultiQuesMarks += countMulQues(text)
	MultiStopMarks += countMulStop(text)

	#print(MultiExclMarks,MultiQuesMarks,MultiStopMarks)

	#print("\nremove multiexclamations from tweet\n")
	text = replaceMulExcl(text)
	#print(text)

	#print("\nremove multiquestionmarks from tweet\n")
	text = replaceMulQues(text)
	#print(text)

	#print("\nremove multistopmarks from tweet\n")
	text = replaceMulStop(text)
	#print(text)


	#print("\nshortening elongated words\n")
	totalElongated = 0
	totalElongated += countElongated(text)
	#print(totalElongated)

	regex1 = re.compile(r"(.)\1{2}")
	l=[]
	for word in text.split():
		if(regex1.search(word)):
			new_word = replaceElongated(word)
			##print(new_word)
			l.append(new_word)
		else:
			l.append(word)
	text = ' '.join(l)
	#print(text)

	#print("\nRemoving punctuations except ?!\n")
	text = removePuncts(text)
	#print(text)
	
	#print("\nexpanding slangs in tweet\n")
	text = replaceSlang(text)
	#print(text)

	#print("\nreplace contractions in tweet\n")
	text = replaceContraction(text)
	#print(text)
	
	#print(\nTokenizing the text\n")
	text = word_tokenize(text)
	
	#print("\nLemmatizing the text\n")
	lemma = WordNetLemmatizer()
	
	list1 = []
	for txt in text:
		list1.append(lemma.lemmatize(txt))
		
	return list1

In [3]:
import pandas as pd
import gensim

df = pd.read_csv("../dataset/train.csv")
df['text'] = df['text'].apply(k_prep)
print(df['text'])

0       [Our, Deeds, are, the, Reason, of, this, earth...
1           [Forest, fire, near, La, Ronge, Sask, Canada]
2       [All, resident, asked, to, shelter, in, place,...
3       [people, receive, wildfire, evacuation, order,...
4       [Just, got, sent, this, photo, from, Ruby, Ala...
                              ...                        
7608    [Two, giant, crane, holding, a, bridge, collap...
7609    [atUser, atUser, The, out, of, control, wild, ...
7610    [M, UTC, ?, km, S, of, Volcano, Hawaii, <, url...
7611    [Police, investigating, after, an, e, bike, co...
7612    [The, Latest, More, Homes, Razed, by, Northern...
Name: text, Length: 7613, dtype: object


In [35]:
# importing the w2c model
# use this or below one
model = gensim.models.KeyedVectors.load_word2vec_format('../../../temp/GoogleNews-vectors-negative300.bin', binary=True)  

In [4]:
# using glove instead fo w2c
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.models.keyedvectors import KeyedVectors
#glove_file = datapath('../../../temp/glove/glove.twitter.27B.25d.txt')
#tmp_file = get_tmpfile("test_word2vec.txt")
#glove2word2vec(glove_file, tmp_file)
glove2word2vec(glove_input_file="../../../temp/glove/glove.twitter.27B.200d.txt", word2vec_output_file="gensim_glove_vectors.txt")
model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt")
model['go']

array([ 2.4313e-01,  1.1597e-02,  3.8135e-01, -5.8350e-01,  1.4441e-01,
        5.6247e-01,  5.2358e-01,  2.0859e-01,  3.6430e-01,  1.6414e-01,
        2.1116e-01, -8.2224e-02, -3.4245e-01, -4.9223e-01, -4.7266e-01,
       -4.2001e-01, -2.2613e-01, -5.3931e-01, -7.0124e-02, -2.2233e-01,
        9.9192e-02, -6.3968e-01,  1.3386e-03, -1.5225e-01,  3.8554e-02,
        5.5047e-01,  1.2805e-01, -1.2323e-01,  3.9681e-01, -2.5109e-01,
       -4.2492e-01,  2.4405e-01, -2.8350e-01,  5.3093e-02,  1.5388e-01,
        7.1608e-01,  4.7376e-01, -4.0382e-02,  2.3105e-01,  2.2754e-01,
        3.2284e-01,  2.8145e-01,  2.5730e-01, -2.0180e-01,  2.6585e-01,
        3.7243e-01,  1.4099e-02, -1.3031e-01,  1.3321e-01,  5.6750e-01,
       -3.4187e-01, -1.6419e-01, -1.3811e-01,  3.5900e-01, -2.5174e-01,
       -7.1623e-02, -4.2692e-01, -2.7603e-02,  3.3421e-01,  5.4517e-01,
        2.5155e-01,  6.7958e-01,  1.4540e-02, -1.2633e-01, -1.2560e-01,
       -4.1516e-01, -5.4850e-01,  8.1463e-02, -1.8129e-01,  6.34

In [36]:

def document_vector(model,doc):
	#filename = 'glove.twitter.27B.25d.txt.word2vec'
	#model = KeyedVectors.load_word2vec_format(filename, binary=False)
	# remove out-of-vocabulary words
	doc = [word for word in doc if word in model.vocab]
	return np.mean(model[doc], axis=0)

In [37]:
new_df = pd.DataFrame()
final_df = []
missed = []
l = []
for index, row in df.iterrows():
    string1 = row['text']
    #l.append(string1)
    try:
        result = document_vector(model, string1)
        #print(result)
        l.append(list(result))
        #print(l)
        #final_df.append(l)
        #a_series = pd.Series(l)
        #print(l)

        #new_df = new_df.append(a_series,ignore_index=True)
    except Exception as e:
        list1 = [
            0.69671315,
            0.049782764,
            -0.24523668,
            -0.15872465,
            -0.0665417,
            0.20241983,
            0.077576466,
            1.9189811,
            -0.006817713,
            0.06951927,
            -0.4152605,
            -0.89838743,
            -3.7327635,
            -0.049629666,
            -0.7617386,
            -0.5561758,
            -0.9451503,
            0.035365578,
            -1.0393411,
            0.2922259,
            -0.16664228,
            -0.46666014,
            0.35039356,
            0.40681368,
            0.38142973,
            ]
        l.append(create_list(300))
        #missed.append(str(index) + ' ')
        #print(e)
        #final_df.append(l)
        #a_series = pd.Series(l)
        #new_df.append(a_series,ignore_index=True)
print(l[:3])
len(l[1])

[[0.021657309, 0.07887014, 0.07608032, 0.08251953, -0.11894735, 0.011985779, -0.0075937905, -0.057469685, 0.057718914, 0.16297404, -0.07683054, -0.005299886, 0.0073140464, 0.08076986, -0.07671102, 0.036931355, 0.098749794, 0.042013805, -0.027061462, -0.029106775, 0.05234782, 0.07655843, 0.16868591, -0.061735153, 0.04561615, 0.054016113, -0.0368042, -0.029675802, -0.0004310608, -0.10961914, -0.02190272, -0.017323812, -0.10821533, -0.09498882, -7.3750816e-05, 0.047477085, -0.046407063, -0.003672282, 0.005845388, 0.09196981, 0.13048808, -0.036346436, 0.116999306, 0.10116577, 0.07077026, -0.022185007, -0.0032170613, 0.043485004, -0.07024256, 0.07159487, -0.041610718, 0.109817505, -0.015406291, 0.029388428, 0.022155762, 0.06945547, -0.09511312, -0.01908048, -0.034551937, -0.055216473, -0.03354009, 0.0558726, -0.10597229, -0.07423655, -0.016693115, -0.068384804, -0.031606037, 0.07804362, -0.01600647, -0.019417444, -0.022867838, -0.025756836, -0.010467529, -0.027395884, -0.083039604, -0.04373

300

In [38]:
# Create the pandas DataFrame 
label_df = df['target']
result_df = pd.DataFrame(l, columns = create_number(301))
#print(result_df)
print(label_df)

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64


In [39]:
print(result_df)

             1         2         3         4         5         6         7  \
0     0.021657  0.078870  0.076080  0.082520 -0.118947  0.011986 -0.007594   
1     0.077427  0.018101  0.116699  0.069196  0.036717 -0.041347 -0.107396   
2    -0.014340 -0.006574  0.084782 -0.012732 -0.063611 -0.023063  0.054099   
3     0.177699 -0.047241  0.004203  0.100237  0.032628 -0.004854 -0.087019   
4     0.086696  0.004797 -0.040091  0.059727 -0.036028 -0.023974 -0.019771   
...        ...       ...       ...       ...       ...       ...       ...   
7608 -0.044428 -0.030121 -0.004439  0.026434  0.019830 -0.139926  0.021290   
7609  0.053911  0.081741  0.038830  0.034012 -0.044636 -0.060463 -0.003998   
7610 -0.170639 -0.028827  0.004154  0.250061  0.020477 -0.069992 -0.123230   
7611  0.041909  0.047636  0.053714 -0.044213 -0.061146  0.001296  0.023490   
7612  0.012226 -0.009672 -0.006827 -0.012755  0.018559  0.037983  0.033324   

             8         9        10  ...       291       292    

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(result_df,label_df, test_size=0.3)

In [41]:
from sklearn.linear_model import RidgeClassifier,LogisticRegression,SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,VotingClassifier
mymodel = RidgeClassifier()
lgr = LogisticRegression()
mnb = MultinomialNB()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()
sgd = SGDClassifier()

mymodel.fit(X_train, y_train)
y_pred = mymodel.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.7907180385288967


In [23]:
#feed to model

In [43]:
#voting
modelnames = [('LogisticRegression',lgr),
		('SGDClassifier',sgd),
		('Gradientboosting',gbc),
		('RandomForest',rfc),
		('RidgeClassifier',mymodel)]

vc = VotingClassifier(voting = 'hard',estimators=modelnames)
vc.fit(X_train,y_train)
y_pred = vc.predict(X_test)
print(accuracy_score(y_test, y_pred))
        

/home/kiranbal/anaconda3/envs/mlvenv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kiranbal/anaconda3/envs/mlvenv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7955341506129597


In [10]:
def create_number(num):
    mylist = []
    for i in range(1,num):
        mylist.append(str(i))
    return mylist


In [82]:
#stacking
from sklearn.ensemble import StackingClassifier
sc = StackingClassifier(estimators=modelnames)
sc.fit(X_train,y_train)
y_pred = sc.predict(X_test)
print(accuracy_score(y_test,y_pred))


ImportError: cannot import name 'StackingClassifier' from 'sklearn.ensemble' (/home/kiranbal/anaconda3/envs/mlvenv/lib/python3.7/site-packages/sklearn/ensemble/__init__.py)

In [ ]:
!pip install mlxtend

In [28]:
def create_list(n):
    import random
    randomlist = []
    for i in range(0,n):
        n = random.uniform(-2,2)
        randomlist.append(n)
    return randomlist